In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

100%|██████████| 25.7M/25.7M [00:00<00:00, 129MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1


In [2]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [3]:
import pandas as pd

df = pd.read_csv("/root/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1/IMDB Dataset.csv")
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:
import re
def preproc(x):
    x = re.sub(r"","", x)
    x = re.sub(r"[^a-zA-Z0-9\s]", "", x)
    x = x.lower()
    return x
df["review"] = df["review"].apply(preproc)
df["sentiment"].replace({"positive" : 1, "negative" : 0}, inplace=True)

<ipython-input-4-39b8090351b2>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["sentiment"].replace({"positive" : 1, "negative" : 0}, inplace=True)
<ipython-input-4-39b8090351b2>:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["sentiment"].replace({"positive" : 1, "negative" : 0}, inplace=Tr

In [5]:
def tok(sentence):
    return tokenizer.encode(sentence, add_special_tokens=True)
df["review"] = df["review"].apply(tok)
df

Token indices sequence length is longer than the specified maximum sequence length for this model (616 > 512). Running this sequence through the model will result in indexing errors


,review,sentiment
0,"[101, 2028, 1997, 1996, 2060, 15814, 2038, 385...",1
1,"[101, 1037, 6919, 2210, 2537, 7987, 7987, 1996...",1
2,"[101, 1045, 2245, 2023, 2001, 1037, 6919, 2126...",1
3,"[101, 10468, 2045, 2015, 1037, 2155, 2073, 103...",0
4,"[101, 9004, 3334, 4717, 17580, 2293, 1999, 199...",1
...,...,...
49995,"[101, 1045, 2245, 2023, 3185, 2106, 1037, 2091...",1
49996,"[101, 2919, 5436, 2919, 7982, 2919, 3772, 1004...",0
49997,"[101, 1045, 2572, 1037, 3234, 4036, 1999, 2877...",0
49998,"[101, 10047, 2183, 2000, 2031, 2000, 21090, 20...",0


In [6]:
max_len = 200
new_data = []

for i in range(len(df)):
    full_review = df.iloc[i, 0]
    sentiment = df.iloc[i, 1]

    if len(full_review) > max_len:
        # Split into chunks of size max_len
        chunks = [full_review[j:j+max_len] for j in range(0, len(full_review), max_len)]

        # Append all chunks to new_data
        for chunk in chunks:
            new_data.append([chunk, sentiment])
    else:
        new_data.append([full_review, sentiment])

# Convert back to DataFrame
df1 = pd.DataFrame(new_data, columns=["review", "sentiment"])


In [7]:
# df1 = pd.DataFrame(l, columns=["review", "sentiment"])
df1


,review,sentiment
0,"[101, 2028, 1997, 1996, 2060, 15814, 2038, 385...",1
1,"[3632, 2073, 2060, 3065, 2876, 2102, 8108, 529...",1
2,"[101, 1037, 6919, 2210, 2537, 7987, 7987, 1996...",1
3,"[101, 1045, 2245, 2023, 2001, 1037, 6919, 2126...",1
4,"[101, 10468, 2045, 2015, 1037, 2155, 2073, 103...",0
...,...,...
86861,"[101, 1045, 2572, 1037, 3234, 4036, 1999, 2877...",0
86862,"[10760, 3268, 1997, 1996, 11113, 15613, 10834,...",0
86863,"[101, 10047, 2183, 2000, 2031, 2000, 21090, 20...",0
86864,"[2011, 2028, 2004, 1996, 2466, 11320, 11140, 2...",0


In [10]:
def pad_sequence(seq, max_len):
    return seq[:max_len] + [0] * (max_len - len(seq))

df1["review"] = df1["review"].apply(lambda x: pad_sequence(x, max_len))

In [12]:
df1.to_csv("data.csv", index=False)

In [11]:
(df1["review"].apply(len))

,review
0,200
1,200
2,200
3,200
4,200
...,...
86861,200
86862,200
86863,200
86864,200
